# Python Script to create dataset file for Championship matches (season 16/17)

This script will create a dataset containing all league one fixtures for the 2016/17 season, and the features for each of those matches.

#### Feature List
1. Date : date of match HomeTeam
2. Home Team AwayTeam
3. Away Team Day_Eve : Is game a day or evening match ? 
4. Day Type : Is the game on a weekend or during week ? 
5. Holiday : Is the game played on a bank holiday ? 
6. Hol Type : Same as Holiday 
7. Capacity : Capacity of home teams ground 
8. Average Travelling Fans : Average number of travelling fans that away team takes (based on previous season) 
9. Cheapest Season T : Lowest Season ticket price for home team 
10. Home League Position : Current position at time of game, of home team 
11. Away League Position : Current position at time of game, of away team 
12. Form Home : Current form of the home team (based on last 5 matches) 
13. Form Away : Current form of the away team (based on last 5 matches) 
14. Distance : Distance between the home sides ground and the away team 
15. Temperature : Temperature on day of game Weather Event : ?? 
16. Lowest Home Ticket Price : Lowest ticket price for a home fan 
17. Lowest Away Ticket Price : Lowest ticket price for an away fan 
18. Home PostCode : Postcode of home team 
19. Away PostCode : Postcode of away team 
20. Attendance : Attendance for the game 
21. Highest Home Ticket Price : Highest home ticket price that a fan can pay

### import relevant python libraries

In [12]:
from pygeocoder import Geocoder
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
from geopy.distance import vincenty
geolocator = Nominatim()

# from datetime import datetime
from datetime import date
from datetime import timedelta
from datetime import datetime
import calendar
import datetime
import holidays
uk_holidays = holidays.UnitedKingdom()

### Creation of Dictionaries related to Teams addresses, ticket prices, ground capacities, away fans averages

In [13]:
# setting the home teams address/postcode if needed
teams = {"Fulham": "Craven Cottage",
         "Birmingham": "Birmingham, B9 4NH",
         "Blackburn": "Ewood Park, Blackburn, BB2 4JF",
         "Bristol City": "Ashton Road, Bristol",
         "Derby": "Pride Park Stadium, Derby",
         "Huddersfield": "Stadium Way, Huddersfield, HD1 6PG",
         "Ipswich": "Portman Road, Ipswich",
         "Nottm Forest": "Nottingham NG2 5FJ",
         "Reading": "Reading, RG2 0FL",
         "Rotherham": "New York Stadium, Rotherham",
         "QPR": "Loftus Road Stadium, South Africa Road, London",
         "Sheffield Wednesday": "Sheffield, S6 1SW",
         "Newcastle": "St James Park, Newcastle",
         "Cardiff": "Cardiff City, CF10 2DS",
         "Norwich City": "Carrow Road, Norwich",
         "Wigan Athletic": "Wigan, WN5 0UZ",
         "Brighton & Hove Albion": "Village Way, Brighton",
         "Brentford": "TW8 0NT",
         "Barnsley": "Barnsley, S71 1ET",
         "Burton Albion": "Princess Way, DE13 0AR",
         "Preston": "Sir Tom Finney Way, Preston",
         "Wolves": "Douglas Turner Way, Wolverhampton",
         "Leeds United": "Elland Road, Leeds",
         "Aston Villa": "Trinity Road, Birmingham"}

# setting capacities of all teams home grounds
capacities = {"Fulham": "25700",
         "Birmingham": "30016",
         "Blackburn": "31367",
         "Bristol City": "21497",
         "Derby": "33597",
         "Huddersfield": "24500",
         "Ipswich": "30311",
         "Nottm Forest": "30602",
         "Reading": "24161",
         "Rotherham": "12021",
         "QPR": "18439",
         "Sheffield Wednesday": "39732",
         "Newcastle": "52404",
         "Cardiff": "28000",
         "Norwich City": "26034",
         "Wigan Athletic": "25133",
         "Brighton & Hove Albion": "30750",
         "Brentford": "12300",
         "Barnsley": "23009",
         "Burton Albion": "6912",
         "Preston": "24500",
         "Wolves": "30852",
         "Leeds United": "37890",
         "Aston Villa": "42788"}

awayfans = {"Fulham": "982",
         "Birmingham": "1841",
         "Blackburn": "1125",
         "Bristol City": "1233",
         "Derby": "2137",
         "Huddersfield": "1095",
         "Ipswich": "1384",
         "Nottm Forest": "1581",
         "Reading": "965",
         "Rotherham": "1026",
         "QPR": "1158",
         "Sheffield Wednesday": "2521",
         "Newcastle": "3140",
         "Cardiff": "775",
         "Norwich City": "1661",
         "Wigan Athletic": "1284",
         "Brighton & Hove Albion": "1611",
         "Brentford": "1062",
         "Barnsley": "1177",
         "Burton Albion": "594",
         "Preston": "1437",
         "Wolves": "1435",
         "Leeds United": "2829",
         "Aston Villa": "3413"}

low_mticket = {"Fulham": "25",
         "Birmingham": "15",
         "Blackburn": "18",
         "Bristol City": "25",
         "Derby": "17.6",
         "Huddersfield": "15",
         "Ipswich": "30",
         "Nottm Forest": "18",
         "Reading": "20",
         "Rotherham": "25",
         "QPR": "29",
         "Sheffield Wednesday": "20",
         "Newcastle": "24",
         "Cardiff": "18",
         "Norwich City": "20",
         "Wigan Athletic": "22",
         "Brighton & Hove Albion": "25",
         "Brentford": "20",
         "Barnsley": "25",
         "Burton Albion": "20",
         "Preston": "24",
         "Wolves": "25",
         "Leeds United": "30",
         "Aston Villa": "20"}

low_away_mticket = {"Fulham": "20",
         "Birmingham": "15",
         "Blackburn": "22",
         "Bristol City": "25",
         "Derby": "17.6",
         "Huddersfield": "20",
         "Ipswich": "10",
         "Nottm Forest": "18",
         "Reading": "20",
         "Rotherham": "23",
         "QPR": "28",
         "Sheffield Wednesday": "20",
         "Newcastle": "24",
         "Cardiff": "22",
         "Norwich City": "20",
         "Wigan Athletic": "20",
         "Brighton & Hove Albion": "25",
         "Brentford": "20",
         "Barnsley": "23",
         "Burton Albion": "20",
         "Preston": "24",
         "Wolves": "30",
         "Leeds United": "35",
         "Aston Villa": "25"}

high_away_mticket = {"Fulham": "30",
         "Birmingham": "30",
         "Blackburn": "30",
         "Bristol City": "30",
         "Derby": "33",
         "Huddersfield": "30",
         "Ipswich": "40",
         "Nottm Forest": "35",
         "Reading": "23",
         "Rotherham": "25",
         "QPR": "34",
         "Sheffield Wednesday": "42",
         "Newcastle": "32",
         "Cardiff": "26",
         "Norwich City": "40",
         "Wigan Athletic": "30",
         "Brighton & Hove Albion": "32",
         "Brentford": "35",
         "Barnsley": "38",
         "Burton Albion": "24",
         "Preston": "24",
         "Wolves": "30",
         "Leeds United": "37",
         "Aston Villa": "35"}

high_mticket = {"Fulham": "45",
         "Birmingham": "40",
         "Blackburn": "35",
         "Bristol City": "41",
         "Derby": "33",
         "Huddersfield": "30",
         "Ipswich": "40",
         "Nottm Forest": "35",
         "Reading": "35",
         "Rotherham": "27",
         "QPR": "38",
         "Sheffield Wednesday": "49",
         "Newcastle": "37",
         "Cardiff": "34",
         "Norwich City": "40",
         "Wigan Athletic": "30",
         "Brighton & Hove Albion": "42",
         "Brentford": "35",
         "Barnsley": "38",
         "Burton Albion": "24",
         "Preston": "30",
         "Wolves": "30",
         "Leeds United": "42",
         "Aston Villa": "37"}

low_season = {"Fulham": "254",
         "Birmingham": "230",
         "Blackburn": "279",
         "Bristol City": "299",
         "Derby": "319",
         "Huddersfield": "179",
         "Ipswich": "417",
         "Nottm Forest": "369",
         "Reading": "325",
         "Rotherham": "355",
         "QPR": "400",
         "Sheffield Wednesday": "415",
         "Newcastle": "428",
         "Cardiff": "299",
         "Norwich City": "499.5",
         "Wigan Athletic": "179",
         "Brighton & Hove Albion": "475",
         "Brentford": "369",
         "Barnsley": "270",
         "Burton Albion": "255",
         "Preston": "360",
         "Wolves": "345",
         "Leeds United": "398",
         "Aston Villa": "325"}

### creation of list containing all league one teams

In [14]:
cols= ["Fulham" , "Birmingham", "Blackburn", "Bristol City", "Derby", "Huddersfield", "Ipswich",
         "Nottm Forest", "Reading", "Rotherham", "QPR", "Sheffield Wednesday", "Newcastle",
         "Cardiff", "Norwich City", "Wigan Athletic", "Brighton & Hove Albion", "Brentford",
         "Barnsley", "Burton Albion", "Preston", "Wolves", "Leeds United", "Aston Villa"]

### Import Source datasets

#### championship_16_17_v2.csv :- 
This contains all league one matches for 2016/17 season (ie. dates, home team, away team). It also contains all the features for each game. These are set initially to "?" and will be populated by the rest of this script.

#### weather.csv
Contains temperatures for all days throughout football season

#### dist_matrix.csv
Contains distances between all football clubs in league one

#### results_championship_16_17_v2.csv
Contains the results of all matches in championship, season 2016/17

#### attendances_championship_16_17.xlsx
Listing of all matches from championship (2016/17 season) and the attendances.

#### league_position_16_17_v2.csv
League position of teams at various stages of season

In [15]:
# Importing the main dataset
dataset = pd.read_csv('championship_16_17_v2.csv')
# Importing the csv file that contains the Weather details
w_data = pd.read_csv('weather.csv')
# Importing the csf file that contains the distance grid for teams
d_data = pd.read_csv('dist_matrix.csv')
# Importing the csf file that contains the results data for teams so can workout form guide etc
f_data = pd.read_csv('results_championship_16_17_v2.csv')
# Importing the league position excel
lp_data = pd.read_csv('league_position_16_17_v2.csv')
# Importing the Attendance file for L1 matches
att_data = pd.read_excel('attendances_championship_16_17.xlsx')

### Counts of records in all source files

In [16]:
#count the number of records in the main csv dataset file
row_cnt = dataset.shape[0]
col_cnt = dataset.shape[1]
print("Records in main Data Set");
print("Row Count = " + str(row_cnt));
print("Column Count = " + str(col_cnt));
print("-----------------------------------");

# count the number of records in the weather dataset
w_r_cnt = w_data.shape[0]
w_col_cnt = w_data.shape[1]
print("Records in Weather Data Set");
print("Row Count = " + str(w_r_cnt));
print("Column Count = " + str(w_col_cnt));
print("-----------------------------------");

# count the number of records in the dist_matrix dataset
d_r_cnt = d_data.shape[0]
d_col_cnt = d_data.shape[1]
print("Records in Distance Data Set");
print("Row Count = " + str(d_r_cnt));
print("Column Count = " + str(d_col_cnt));
print("-----------------------------------");

# count the number of records in the championship results set dataset
f_r_cnt = f_data.shape[0]
f_col_cnt = f_data.shape[1]
print("Records in Results Championship Data Set");
print("Row Count = " + str(f_r_cnt));
print("Column Count = " + str(f_col_cnt));
print("-----------------------------------");

# count the number of records in the league position dataset
lp_r_cnt = f_data.shape[0]
lp_col_cnt = f_data.shape[1]
print("Records in League Position Data Set");
print("Row Count = " + str(lp_r_cnt));
print("Column Count = " + str(lp_col_cnt));
print("-----------------------------------");

Records in main Data Set
Row Count = 552
Column Count = 23
-----------------------------------
Records in Weather Data Set
Row Count = 7063
Column Count = 6
-----------------------------------
Records in Distance Data Set
Row Count = 24
Column Count = 26
-----------------------------------
Records in Results Championship Data Set
Row Count = 552
Column Count = 6
-----------------------------------
Records in League Position Data Set
Row Count = 552
Column Count = 6
-----------------------------------


### For all rows in the main data set, calculate all features (loop structure)
- Section 1: Distance Calculation
- Section 2: Day of Week Calculation
- Section 3: Assign Capacity of Ground
- Section 4: Weather
- Section 5: Form Calculation of teams
- Section 6: Average Away fans following
- Section 7: Lowest Matchday prices (Home Team)
- Section 8: Highest Matchday prices (Home Team)
- Section 9: Lowest Matchday prices (Away Team)
- Section 10: Cheapest Season Ticket price for home team
- Section 11: League positions of each team playing in match
- Section 12: Set the Attendance figure for match

In [17]:
# loop to calculate new values of dataset and then write to new file
# ------------------------------------------------------------------
for rows in range(0,row_cnt):
# for rows in range(0,150):
    
    # Section 1 - Distance Calculation
    # calculate the distance of teams playing in miles
    # rows,1 = 2nd column, as first column is 0 (1st col = home team)
    # print("***STARTING LOOP***");
    # print(dataset.iloc[rows,1]);   
    home_team = dataset.iloc[rows,1]
    away_team = dataset.iloc[rows,2]
    print("Home Team = " + home_team);
    print("Away Team = " + away_team);
    rowds = pd.DataFrame(d_data[d_data.HomeTeam == home_team])
    distance=int(rowds[away_team])
    # print("Distance between teams = " + str(distance));
    # rows,14 = 15th column, ie. distance value between teams in miles
    dataset.iloc[rows,14]=str(distance)
    
    # Section 2 - Day of Week Calculation
    # calculate the day of the week for the match
    # print("Distance between teams = " + dataset.iloc[rows,14]);
    # print("Match Date = " + dataset.iloc[rows,0]);
    # stripping out time from match date, and getting day of week
    m_date=datetime.datetime.strptime(dataset.iloc[rows,0], '%d/%m/%Y')
    m_day_of_week=calendar.day_name[m_date.weekday()]
    # day type = 0 if Sat or Sun, else its 1.
    # day_eve value is D if Sat or Sun, else its E (evening)
    if m_day_of_week=="Saturday" or m_day_of_week=="Sunday":
        day_eve="D"
        day_type="0"
    else:
        day_eve="E"
        day_type="1"
    # print(m_date);
    # print("Day of the week = " + m_day_of_week);
    # print("day_eve = " + day_eve);
    # print("day_type = " + day_type);
    # rows,3 (4th column) = day_eve value, set the new value
    # rows,4 (5th column) = day_type value, set the new value
    dataset.iloc[rows,3]=str(day_eve)
    dataset.iloc[rows,4]=str(day_type)
    
    # Section 3 - Assign Capacity of ground to each match
    # assign capacities to home grounds/matches
    home_cap=capacities.get(dataset.iloc[rows,1])
    print(home_cap);
    dataset.iloc[rows,7]=str(home_cap)
    
    # Section 4 - Get the weather / temperature on day of game
    # get weather / temperature from day of game, utilising the w_data dataset
    # rows,15 = temperature column in main datset
    chk_date = datetime.datetime.strptime(dataset.iloc[rows,0],"%d/%m/%Y")
    # print("Find Temperature for home team, on match date")
    # print("Match Date = " + chk_date.strftime("%Y-%m-%d"));
    # print("home team = " + dataset.iloc[rows,1]);
    # setting up a new ds for home team
    home_set = w_data.loc[w_data['team'] == dataset.iloc[rows,1]];   
    # now using the home_set dataset, find the temperature on match date
    temp_flag="N"
    while temp_flag == "N":
        home_temp = pd.DataFrame(home_set.loc[home_set['Date'] == chk_date.strftime("%Y-%m-%d")])
        if home_temp.empty:
            print("DataFrame is Empty, minus 1 day")
            chk_date = chk_date - timedelta(days=1)
            print(chk_date);
        else:
            temp_flag="Y"
    temp_1 = home_temp.iloc[0,3]
    # print(temp_1);
    dataset.iloc[rows,15]=str(temp_1)
        
    # Section 5 - Form of Teams Calculation - using last 5 games
    # Last 5 games, form guide !!
    # print("--------------------------");
    tmp_date = dataset.iloc[rows,0]
    # print("Date of match = " + tmp_date);
    # print("Home Team = " + home_team)
    # print("Away Team = " + away_team)   

    # check date to see if its a holiday
    # use the holidays package
    if (tmp_date in uk_holidays):
        dataset.iloc[rows,6] = 1
    else:
        dataset.iloc[rows,6] = 0
    
    # Setting up data sets so that we can work out the form, for 5 matches before
    home_form = f_data.loc[(f_data['Home'] == home_team) | (f_data['Away'] == home_team)]
    away_form = f_data.loc[(f_data['Home'] == away_team) | (f_data['Away'] == away_team)]
    # resetting index of each data set, so ordered fromn 0 upwards in increment of 1
    home_form = home_form.reset_index(drop=True)
    away_form = away_form.reset_index(drop=True)
    # calculate index position of row which matches the home team
    idx_h_position = home_form[home_form['New_Date'] == tmp_date].index.tolist()
    # print("idx_h_position = " + str(idx_h_position))
    # get maximum value of index
    max_h = len(home_form.index)-1
    # print("max_h = " + str(max_h));
    # calculate index position of row which matches the away team
    idx_a_position = home_form[home_form['New_Date'] == tmp_date].index.tolist()
    # print("idx_a_position = " + str(idx_a_position))
    # get maximum value of index
    max_a = len(away_form.index)-1
    # print("max_a = " + str(max_a));
    
    # now work out the points scored in each of the 5 previous games
    # this is done by looking at the previous 5 games in the dataset using
    # the index, idx_h_position
    pts_form = 0
    for i in range(idx_h_position[0]+1,min(idx_h_position[0]+6,max_h)+1):
        # print(home_form.iloc[i].home_score);
        if (home_form.iloc[i].Home == home_team) & (home_form.iloc[i].home_score > home_form.iloc[i].away_score):
            pts_form = pts_form + 3
        if (home_form.iloc[i].Away == home_team) & (home_form.iloc[i].away_score > home_form.iloc[i].home_score):
            pts_form = pts_form + 3    
        if home_form.iloc[i].home_score == home_form.iloc[i].away_score:
            pts_form = pts_form + 1            
        # print("Home form pts = " + str(pts_form))
    # set the home form value on main dataset
    dataset.iloc[rows,12]=str(pts_form)
    pts_form = 0
    for i in range(idx_a_position[0]+1,min(idx_a_position[0]+6,max_a)+1):
        # print(away_form.iloc[i].away_score);
        if (away_form.iloc[i].Away == away_team) & (away_form.iloc[i].away_score > away_form.iloc[i].home_score):
            pts_form = pts_form + 3
        if (away_form.iloc[i].Home == away_team) & (away_form.iloc[i].home_score > away_form.iloc[i].away_score):
            pts_form = pts_form + 3
        if away_form.iloc[i].home_score == away_form.iloc[i].away_score:
            pts_form = pts_form + 1            
        # print("Away form pts = " + str(pts_form))
    # set the away form value on main dataset
    dataset.iloc[rows,13]=str(pts_form)
    pts_form = 0
    
    # Section 6 - Setting of Avg Away Fans following/number for each game
    # now set the away fans following feature for each match
    away_following=awayfans.get(dataset.iloc[rows,2])
    # print(away_following);
    dataset.iloc[rows,8]=str(away_following)
    
    # Section 7 - Set the value of the Lowest Match Day Prices for HOME team, for each game
    # now set the lowest match day prices feature for each match for each match (home team)
    low_tick=low_mticket.get(dataset.iloc[rows,1])
    # print("Lowest Matchday Ticket Price (home) = " + low_tick);
    dataset.iloc[rows,17]=str(low_tick)
    
    # Section 8 - Set the highest matchday prices for HOME team, for each game
    # now set the highest match day prices feature for each match (home team)
    high_tick=high_mticket.get(dataset.iloc[rows,1])
    # print("Highest Matchday Ticket Price (home) = " + high_tick);
    dataset.iloc[rows,22]=str(high_tick)
    
    # Section 9 - Set the lowest matchday prices for AWAY team, for each game
    # now set the lowest match day prices feature for each match (away team)
    low_a_tick=low_away_mticket.get(dataset.iloc[rows,2])
    # print("Lowest Matchday Ticket Price (away) = " + low_a_tick);
    dataset.iloc[rows,18]=str(low_a_tick)
    
    # Section 10 - Set the cheapeast season ticket value for each match
    # now set the cheapest season ticket feature for each match
    low_seas=low_season.get(dataset.iloc[rows,1])
    # print("Lowest Home Season Ticket Price = " + low_seas);
    dataset.iloc[rows,9]=str(low_seas)
    
    # Section 11 - Calculate the league position, at time of game, of each team playing in the match
    # now work out the league position of the teams
    home_position = lp_data.loc[(lp_data['Team'] == dataset.iloc[rows,1]) &
                                (lp_data['Opponents'] == dataset.iloc[rows,2]) &
                                (lp_data['Home/Away'] == 'H')]
    dataset.iloc[rows,10]=str(home_position.iloc[0,6])
    away_position = lp_data.loc[(lp_data['Team'] == dataset.iloc[rows,2]) &
                                (lp_data['Opponents'] == dataset.iloc[rows,1]) &
                                (lp_data['Home/Away'] == 'A')]
    dataset.iloc[rows,11]=str(away_position.iloc[0,6])
    
    # Section 12 - Set the attendance figure for the game
    find_match = att_data[(att_data['home_team'].str.strip() == home_team) & (att_data['away_team'].str.strip() == away_team)]

    if not(find_match.empty):
        print("Attendance Setting");
        dataset.iloc[rows,21] = find_match.iloc[0,0]
        
    # End of loop
    print("+++++++++++++++++++++++++++++");

Home Team = Fulham
Away Team = Newcastle
25700
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Birmingham
Away Team = Cardiff
30016
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Blackburn
Away Team = Norwich City
31367
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Bristol City
Away Team = Wigan Athletic
21497
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Derby
Away Team = Brighton & Hove Albion
33597
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Huddersfield
Away Team = Brentford
24500
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Ipswich
Away Team = Barnsley
30311
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Nottm Forest
Away Team = Burton Albion
30602
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Reading
Away Team = Preston
24161
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Rotherham
Away Team = Wolves
12021
Attendance Setting
++++++++++++++++++++++++

Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Rotherham
Away Team = Nottm Forest
12021
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Barnsley
Away Team = Reading
23009
DataFrame is Empty, minus 1 day
2016-09-16 00:00:00
DataFrame is Empty, minus 1 day
2016-09-15 00:00:00
DataFrame is Empty, minus 1 day
2016-09-14 00:00:00
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Birmingham
Away Team = Sheffield Wednesday
30016
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Blackburn
Away Team = Rotherham
31367
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Brentford
Away Team = Preston
12300
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Bristol City
Away Team = Derby
21497
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Burton Albion
Away Team = Brighton & Hove Albion
6912
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Cardiff
Away Team = Leeds United
28000
Attendance Setting
++++++++

Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Wigan Athletic
Away Team = Brighton & Hove Albion
25133
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Wolves
Away Team = Leeds United
30852
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = QPR
Away Team = Brentford
18439
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Barnsley
Away Team = Bristol City
23009
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Blackburn
Away Team = Wolves
31367
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Brighton & Hove Albion
Away Team = Norwich City
30750
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Cardiff
Away Team = Wigan Athletic
28000
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Derby
Away Team = Sheffield Wednesday
33597
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Fulham
Away Team = Huddersfield
25700
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Ipswich
Away

Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Ipswich
Away Team = Cardiff
30311
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Newcastle
Away Team = Birmingham
52404
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Preston
Away Team = Blackburn
24500
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Reading
Away Team = Sheffield Wednesday
24161
DataFrame is Empty, minus 1 day
2016-12-09 00:00:00
DataFrame is Empty, minus 1 day
2016-12-08 00:00:00
DataFrame is Empty, minus 1 day
2016-12-07 00:00:00
DataFrame is Empty, minus 1 day
2016-12-06 00:00:00
DataFrame is Empty, minus 1 day
2016-12-05 00:00:00
DataFrame is Empty, minus 1 day
2016-12-04 00:00:00
DataFrame is Empty, minus 1 day
2016-12-03 00:00:00
DataFrame is Empty, minus 1 day
2016-12-02 00:00:00
DataFrame is Empty, minus 1 day
2016-12-01 00:00:00
DataFrame is Empty, minus 1 day
2016-11-30 00:00:00
DataFrame is Empty, minus 1 day
2016-11-29 00:00:00
DataFrame is Empty, minus 1 d

Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Leeds United
Away Team = Derby
37890
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Birmingham
Away Team = Nottm Forest
30016
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Brentford
Away Team = Newcastle
12300
DataFrame is Empty, minus 1 day
2017-01-13 00:00:00
DataFrame is Empty, minus 1 day
2017-01-12 00:00:00
DataFrame is Empty, minus 1 day
2017-01-11 00:00:00
DataFrame is Empty, minus 1 day
2017-01-10 00:00:00
DataFrame is Empty, minus 1 day
2017-01-09 00:00:00
DataFrame is Empty, minus 1 day
2017-01-08 00:00:00
DataFrame is Empty, minus 1 day
2017-01-07 00:00:00
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Bristol City
Away Team = Cardiff
21497
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Burton Albion
Away Team = Wigan Athletic
6912
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Fulham
Away Team = Barnsley
25700
Attendance Setting
+++++++++++++++

Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Aston Villa
Away Team = Ipswich
42788
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Brighton & Hove Albion
Away Team = Burton Albion
30750
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Derby
Away Team = Bristol City
33597
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Fulham
Away Team = Wigan Athletic
25700
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Leeds United
Away Team = Cardiff
37890
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Norwich City
Away Team = Nottm Forest
26034
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Preston
Away Team = Brentford
24500
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = QPR
Away Team = Huddersfield
18439
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Reading
Away Team = Barnsley
24161
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Rotherham
Away Team = Blackburn

Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Nottm Forest
Away Team = Brighton & Hove Albion
30602
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = QPR
Away Team = Cardiff
18439
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Reading
Away Team = Wolves
24161
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Rotherham
Away Team = Aston Villa
12021
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Sheffield Wednesday
Away Team = Norwich City
39732
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Birmingham
Away Team = Wigan Athletic
30016
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Blackburn
Away Team = Cardiff
31367
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Bristol City
Away Team = Norwich City
21497
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Derby
Away Team = Preston
33597
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Fulham
Away Team = Leeds U

Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Sheffield Wednesday
Away Team = Reading
39732
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Birmingham
Away Team = Newcastle
30016
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Blackburn
Away Team = Preston
31367
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Burton Albion
Away Team = Brentford
6912
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Cardiff
Away Team = Ipswich
28000
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Fulham
Away Team = Wolves
25700
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Leeds United
Away Team = Brighton & Hove Albion
37890
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Norwich City
Away Team = Barnsley
26034
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Nottm Forest
Away Team = Derby
30602
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = QPR
Away Team = Rotherham
18439

Away Team = Burton Albion
23009
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Birmingham
Away Team = Huddersfield
30016
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Blackburn
Away Team = Aston Villa
31367
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Brighton & Hove Albion
Away Team = Bristol City
30750
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Derby
Away Team = Wolves
33597
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Fulham
Away Team = Brentford
25700
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Ipswich
Away Team = Sheffield Wednesday
30311
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Leeds United
Away Team = Norwich City
37890
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = Preston
Away Team = Rotherham
24500
Attendance Setting
+++++++++++++++++++++++++++++
Home Team = QPR
Away Team = Nottm Forest
18439
Attendance Setting
+++++++++++++++++++++++++++++
Home Te

In [18]:
dataset.to_csv('championship_dataset.csv')